<a href="https://colab.research.google.com/github/Pysrdafoice/Cronograma-de-Estudos-Banco-de-horas/blob/main/Main_cronograma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 #main.py
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# --- Estrutura de Dados em Memória (GLOBAL) ---
# O dicionário 'DADOS_PROJETO' armazenará todos os dados em memória.
DADOS_PROJETO = {
    # 'df_registros' irá armazenar o histórico de estudos (substituindo o CSV)
    'df_registros': pd.DataFrame(columns=['Data', 'Materia', 'Horas_Estudadas', 'Meta_Diaria', 'Saldo_Dia', 'Acumulado_Banco_Horas', 'Penalidade_Aplicada']),

    # 'metas_cursos' irá armazenar as metas de cada matéria (substituindo o JSON)
    # Exemplo: 'Python': {'meta_total_horas': 44.0, 'meta_diaria': 2.0, 'progresso_total': 0.0}
    'metas_cursos': {}
}


def configurar_metas():
    """Permite ao usuário registrar o nome, meta total e meta diária de cada matéria."""
    print("\n--- CONFIGURAÇÃO DE METAS DE ESTUDO ---")
    while True:
        materia = input("Digite o nome da Matéria (ou 'sair' para terminar): ").strip()
        if materia.lower() == 'sair':
            break

        try:
            meta_total = float(input(f"Meta de Horas TOTAL para '{materia}': "))
            meta_diaria = float(input(f"Meta Diária de Horas para '{materia}': "))
        except ValueError:
            print("Entrada inválida. Digite um número para as horas.")
            continue

        # Armazena a meta no dicionário global
        DADOS_PROJETO['metas_cursos'][materia] = {
            'meta_total_horas': meta_total,
            'meta_diaria': meta_diaria,
            'progresso_total': 0.0 # Inicializa o progresso
        }
        print(f"Matéria '{materia}' configurada com sucesso!")

def inicializar_dataframes():
    """
    Função de inicialização única para garantir que o DataFrame de registros
    esteja vazio e pronto para uso em memória.
    """
    if DADOS_PROJETO['df_registros'].empty:
        print("DataFrame de registros inicializado em memória.")
    else:
        # Se houver dados (em caso de reinicialização no runtime, por exemplo),
        # poderíamos carregá-los ou limpar aqui. Por enquanto, apenas avisamos.
        print("Dados de registros já existem em memória.")

def calcular_e_registrar_estudo():
    """
    Função para coletar o input diário, aplicar a lógica do banco de horas
    e registrar o novo estado no DataFrame.
    """
    print("\n--- REGISTRAR ESTUDO DIÁRIO ---")
    if not DADOS_PROJETO['metas_cursos']:
        print("ERRO: Configure as metas das matérias antes de registrar o estudo (Opção 2).")
        return

    # 1. Coleta Inputs do Usuário
    data_str = input("Digite a data do estudo (YYYY-MM-DD): ")
    try:
        data = datetime.strptime(data_str, '%Y-%m-%d').date()
    except ValueError:
        print("Formato de data inválido. Use YYYY-MM-DD.")
        return

    print("Matérias disponíveis:", list(DADOS_PROJETO['metas_cursos'].keys()))
    materia = input("Qual matéria você estudou hoje?: ").strip()

    if materia not in DADOS_PROJETO['metas_cursos']:
        print(f"Matéria '{materia}' não configurada.")
        return

    try:
        horas_estudadas = float(input(f"Quantas horas você estudou {materia} hoje?: "))
    except ValueError:
        print("Valor de horas inválido. Digite um número.")
        return

    meta_diaria = DADOS_PROJETO['metas_cursos'][materia]['meta_diaria']
    saldo_dia = horas_estudadas - meta_diaria

    # 2. Lógica de Penalidade do Banco de Horas (CRÍTICA!)
    # Esta é a parte mais complexa e que precisa ser implementada com rigor.

    df = DADOS_PROJETO['df_registros']
    ultimo_saldo_acumulado = df['Acumulado_Banco_Horas'].iloc[-1] if not df.empty else 0.0
    penalidade_aplicada = 0.0

    # Simulação da verificação de falta (A LÓGICA DE VERIFICAÇÃO DE FALTA POR DIA ÚTIL DEVE SER REFINADA)
    # Por enquanto, assumimos que o registro é para o dia atual, e o cálculo de falta
    # para dias anteriores será feito em uma função de auditoria/recalculo.

    # Se o saldo do dia for positivo, acumula.
    novo_acumulado = ultimo_saldo_acumulado + saldo_dia

    # 3. Cria e Adiciona o Novo Registro
    novo_registro = {
        'Data': data_str,
        'Materia': materia,
        'Horas_Estudadas': horas_estudadas,
        'Meta_Diaria': meta_diaria,
        'Saldo_Dia': saldo_dia,
        'Acumulado_Banco_Horas': novo_acumulado,
        'Penalidade_Aplicada': penalidade_aplicada
    }

    # Atualiza o DataFrame em memória
    novo_registro_df = pd.DataFrame([novo_registro])
    DADOS_PROJETO['df_registros'] = pd.concat([df, novo_registro_df], ignore_index=True)

    # 4. Atualiza a Meta Total da Matéria
    DADOS_PROJETO['metas_cursos'][materia]['progresso_total'] += horas_estudadas

    print("✅ Registro de estudo adicionado e Banco de Horas atualizado!")
    print(f"Novo Saldo Acumulado: {novo_acumulado:.2f} horas")


def visualizar_progresso():
    """Gera um gráfico simples do andamento dos dias e horas estudadas."""
    print("\n--- VISUALIZAÇÃO DE PROGRESSO ---")
    df = DADOS_PROJETO['df_registros']

    if df.empty:
        print("Não há dados para plotar. Registre algum estudo primeiro.")
        return

    # Converte a coluna de Data para o formato datetime para plotagem
    df['Data'] = pd.to_datetime(df['Data'])

    # Agrupa por data, somando as horas estudadas (caso tenha estudado mais de uma matéria no dia)
    progresso_diario = df.groupby('Data')['Horas_Estudadas'].sum().reset_index()

    # Plota o gráfico (Requisito: Dias estudados vs. Horas estudadas)
    plt.figure(figsize=(12, 6))

    # Gráfico de Barras para Horas Estudadas
    plt.bar(progresso_diario['Data'], progresso_diario['Horas_Estudadas'], color='#3b82f6', label='Horas Estudadas')

    # Adiciona a linha da Meta Diária (usando a média das metas diárias configuradas, apenas como referência)
    meta_media = sum(m['meta_diaria'] for m in DADOS_PROJETO['metas_cursos'].values()) / len(DADOS_PROJETO['metas_cursos'])
    plt.axhline(y=meta_media, color='#dc2626', linestyle='--', label=f'Meta Diária Média ({meta_media:.2f}h)')

    plt.title('Andamento Diário de Horas de Estudo', fontsize=16, fontweight='bold')
    plt.xlabel('Dia', fontsize=12)
    plt.ylabel('Horas', fontsize=12)
    plt.legend()
    plt.grid(axis='y', linestyle='dotted', alpha=0.7)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


def main():
    """Função principal do sistema CLI."""

    inicializar_dataframes()

    print("\n--- Bem-vindo ao SCGE (Sistema de Controle e Gráficos de Estudos) ---")
    print("OBS: Todos os dados são armazenados em memória e serão perdidos ao fechar o programa.")

    while True:
        print("\nO que você gostaria de fazer?")
        print("1. Registrar estudo diário")
        print("2. Configurar Metas de Matérias")
        print("3. Visualizar Progresso (Gráfico)")
        print("4. Status do Banco de Horas")
        print("5. Sair")

        escolha = input("Digite a opção (1, 2, 3, 4, 5): ")

        if escolha == '1':
            calcular_e_registrar_estudo()
        elif escolha == '2':
            configurar_metas()
        elif escolha == '3':
            visualizar_progresso()
        elif escolha == '4':
            df = DADOS_PROJETO['df_registros']
            saldo = df['Acumulado_Banco_Horas'].iloc[-1] if not df.empty else 0.0
            print(f"\nSaldo Atual do Banco de Horas: {saldo:.2f} horas")
            print("Progresso das Metas Totais:")
            for materia, dados in DADOS_PROJETO['metas_cursos'].items():
                 print(f"- {materia}: {dados['progresso_total']:.1f}h de {dados['meta_total_horas']:.1f}h")
        elif escolha == '5':
            print("Obrigado por usar o SCGE. Até mais!")
            break
        else:
            print("Opção inválida. Tente novamente.")

if __name__ == "__main__":
    main()


DataFrame de registros inicializado em memória.

--- Bem-vindo ao SCGE (Sistema de Controle e Gráficos de Estudos) ---
OBS: Todos os dados são armazenados em memória e serão perdidos ao fechar o programa.

O que você gostaria de fazer?
1. Registrar estudo diário
2. Configurar Metas de Matérias
3. Visualizar Progresso (Gráfico)
4. Status do Banco de Horas
5. Sair
Digite a opção (1, 2, 3, 4, 5): 34
Opção inválida. Tente novamente.

O que você gostaria de fazer?
1. Registrar estudo diário
2. Configurar Metas de Matérias
3. Visualizar Progresso (Gráfico)
4. Status do Banco de Horas
5. Sair
Digite a opção (1, 2, 3, 4, 5): 4

Saldo Atual do Banco de Horas: 0.00 horas
Progresso das Metas Totais:

O que você gostaria de fazer?
1. Registrar estudo diário
2. Configurar Metas de Matérias
3. Visualizar Progresso (Gráfico)
4. Status do Banco de Horas
5. Sair
Digite a opção (1, 2, 3, 4, 5): um
Opção inválida. Tente novamente.

O que você gostaria de fazer?
1. Registrar estudo diário
2. Configurar Me